In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

try:
    from sklearn.datasets import fetch_openml
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LogisticRegression
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
    from sklearn.neural_network import MLPClassifier
    import seaborn as sns
except ImportError as e:
    import subprocess
    import sys

    print(f"Missing package: {str(e).split()[-1]}")
    print("Installing required packages...")

Part 1:

Follow the steps in the first link below to compute the forward and backward passes of the neural network. Use the 2nd link to assist you with this task (look at the last cell). Use the same weights and inputs and print out the new weights connected to the hidden and output layers. (You should print the exact weight values computed in the first link)

 https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/

https://github.com/snsie/ai-webdev/blob/main/lessons/lesson-02/lesson-02.solutions.ipynb

In [9]:
# create inputs array from example webpage
inputs=[0.05,0.1]

# create outputs array from example webpage
outputs=[0.01,0.99]

# create learning rate from example webpage
lr=0.5

In [16]:
class NnFp:
    """
    Base class for Neural Network Forward Pass
    """
    def __init__(self, inputs, outputs):
        self.inputs = np.array(inputs)
        self.outputs = np.array(outputs)
        
        # Initialize weights as per the example
        self.weightsToHl = np.array([[0.15, 0.20],
                                   [0.25, 0.30],
                                   [0.35, 0.35]])  # Last row is bias
        
        self.weightsToOl = np.array([[0.40, 0.45],
                                   [0.50, 0.55],
                                   [0.60, 0.60]])  # Last row is bias
    
    def sigmoid(self, x, derivative=False):
        if derivative:
            return x * (1 - x)
        return 1 / (1 + np.exp(-x))
    
    def getHlNet(self):
        inputsWithBias = np.array([*self.inputs, 1])
        return inputsWithBias @ self.weightsToHl
    
    def getHlOutputs(self):
        hlNet = self.getHlNet()
        return self.sigmoid(hlNet)
    
    def getOlNet(self):
        hlOutputs = self.getHlOutputs()
        hlOutputsWithBias = np.array([*hlOutputs, 1])
        return hlOutputsWithBias @ self.weightsToOl
    
    def getOlOutputs(self):
        olNet = self.getOlNet()
        return self.sigmoid(olNet)

class NeuralNet(NnFp):
    """
    updates weights via backpropagation
    """
    
    def __init__(self,inputs,outputs, lr):
        super().__init__(inputs,outputs)
        self.lr=lr
    
    # get the change in total error with respect to each ol output
    def get_dE_dOlOut(self):
      predOutputs=self.getOlOutputs()
      targets = self.outputs
      return predOutputs-targets

    def get_dOlOut_dOlNet(self):
      olNet=self.getOlNet()
      return self.sigmoid(olNet,derivative=True)

    def get_dOlNet_dWOl(self):
      hlOutputs=self.getHlOutputs()
      hlOutputsWithBias=np.array([*hlOutputs,1])
      return hlOutputsWithBias.reshape(len(hlOutputsWithBias),1)
      
    def get_dE_dOlNet(self):
      dE_dOlOut=self.get_dE_dOlOut()
      dOlOut_dOlNet=self.get_dOlOut_dOlNet()
      dE_dOlNet=dE_dOlOut*dOlOut_dOlNet
      return dE_dOlNet

    def getPartialWeightsToOl(self):
      dE_dOlNet=self.get_dE_dOlNet()
      dE_dOlNet=dE_dOlNet.reshape(1,len(dE_dOlNet))
      dOlNet_dWOl=self.get_dOlNet_dWOl()
      return dOlNet_dWOl@dE_dOlNet

    def getNextWeightsToOl(self):
      partialWeightsToOl=self.getPartialWeightsToOl()
      nextWeightsToOl=self.weightsToOl-self.lr*partialWeightsToOl
      return nextWeightsToOl

    ########## updating weights to hidden layer below

    def get_dE_dHlOut(self):
      dE_dOlNet=self.get_dE_dOlNet()
      dE_dOlNet=dE_dOlNet.reshape(np.size(dE_dOlNet),1)
      weightsToOlWithoutBias=np.delete(self.weightsToOl,2,0)
      dOlNet_dHlOut=weightsToOlWithoutBias
      return dOlNet_dHlOut@dE_dOlNet

    def get_dHlOut_dHlNet(self):
      hlNet=self.getHlNet() 
      return self.sigmoid(hlNet,derivative=True)

    def get_dHlNet_dWHl(self):
      inputsWithBias = np.array([*self.inputs,1])
      return inputsWithBias.reshape(len(inputsWithBias),1)

    def get_dE_dHlNet(self):
      dE_dHlOut=self.get_dE_dHlOut().flatten()
      dHlOut_dHlNet=self.get_dHlOut_dHlNet()
      return dE_dHlOut*dHlOut_dHlNet

    # create a function to calculate the change in total error with 
    # respect to each weight connected to the hidden nodes
    def getPartialWeightsToHl(self):
      dE_dHlNet=self.get_dE_dHlNet()
      dE_dHlNet=dE_dHlNet.reshape(1,len(dE_dHlNet))
      dHlNet_dWHl=self.get_dHlNet_dWHl()
      return dHlNet_dWHl@dE_dHlNet

    def getNextWeightsToHl(self):
        partialWeightsToHl = self.getPartialWeightsToHl()
        nextWeightsToHl = self.weightsToHl - self.lr * partialWeightsToHl
        print("\nHidden Layer Weights:")
        print(np.array2string(nextWeightsToHl, precision=5, suppress_small=True))
        return nextWeightsToHl

    def getNextWeightsToOl(self):
        partialWeightsToOl = self.getPartialWeightsToOl()
        nextWeightsToOl = self.weightsToOl - self.lr * partialWeightsToOl
        print("\nOutput Layer Weights:")
        print(np.array2string(nextWeightsToOl, precision=5, suppress_small=True))
        return nextWeightsToOl




In [18]:
net = NeuralNet(inputs, outputs, lr)
new_hl_weights = net.getNextWeightsToHl()
new_ol_weights = net.getNextWeightsToOl()


Hidden Layer Weights:
[[0.15013 0.20017]
 [0.25027 0.30035]
 [0.35269 0.35346]]

Output Layer Weights:
[[0.43428 0.43456]
 [0.53439 0.53451]
 [0.65767 0.57403]]


Part 2:

Create a neural network using Numpy and train it on the MNIST dataset. (Cursor will assist you)

Evaluate the accuracy and F1 scores for each of the 10 digits. Use figures to visualize your results.